In [1]:
import pandas as pd
Tenders = pd.read_excel("C:/Users/Mitch/git/UWACapstoneG2/data/Tenders.xlsx")

In [2]:
# unique links
Prep = Tenders["TenderLink"].unique()
# without nan
Links = Prep[~pd.isnull(Prep)]

In [3]:
# Extracts tender zip files from links
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# buttons 
locators = {
    "Download Now": By.LINK_TEXT,
    "Download for Information Only": By.XPATH,
    "Download Documents": By.XPATH
}

def click_button(driver, wait, button_text):
    
    locator = locators[button_text]
    if locator == "xpath":
        button_text = f"//input[@value='{button_text}']"
    
    try:
        button = wait.until(EC.element_to_be_clickable((locator, button_text)))
        button.click()
    except Exception as e:
        print(f"Error while clicking button '{button_text}': {e}")

def get_tenders(link):
    driver = webdriver.Chrome()

    # tender webpage
    driver.get(link)

    # wait for button to be clickable
    wait = WebDriverWait(driver, 10)
    
    # click button in this order
    buttons = ["Download Now", "Download for Information Only", "Download Documents"]
    
    for button in buttons:
        click_button(driver, wait, button)

    # enough time to download files before quitting
    time.sleep(5)
    driver.quit()

In [4]:
get_tenders(Links[0])
#for i in Links:
#    get_tenders(i)